In [1]:
import pandas as pd
import os
import cv2 as cv
import tensorflow as tf
from tensorflow.train import Example , Int64List , BytesList , Features , FloatList , Feature 
import xml.etree.cElementTree as ET
from PIL import Image
from tqdm import tqdm
import lxml.etree
import time
from absl import logging , app , flags
from contextlib import ExitStack
import hashlib

# -----------------------------------
# Create jpg images from ppm images :

In [2]:
data_dir = '../data/TrainIJCNN2013/'
data_info_dir = '../data/TrainIJCNN2013/data_info.csv'
data_info_df = pd.read_csv(data_info_dir)
final_df = pd.DataFrame(columns=data_info_df.columns)

In [3]:
for i in range(len(data_info_df)):
    data_info_df['path'][i] = data_info_df['path'][i][:-4]

<ipython-input-3-16cd3cf7d9f3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_info_df['path'][i] = data_info_df['path'][i][:-4]


In [4]:
for current_dir , dirs ,files in os.walk(data_dir):
    for f in files:
        if f.endswith('.ppm'):
            image_name = f[:-4]
            image = cv.imread(data_dir + f)
            single_data_line = data_info_df.loc[data_info_df['path'] == f[:-4]].copy()
            if single_data_line.isnull().values.all():
                os.remove(data_dir + f)
            else:
                final_df = final_df.append(single_data_line)
                save_path = "../data/Images/" + image_name + '.jpg'
                if not os.path.isfile(save_path):
                    cv.imwrite(save_path,image)

In [5]:
final_df = final_df[~final_df.index.duplicated(keep='first')]
final_df.sort_index(inplace=True)

# -----------------------------------
# Creating annotations files :

In [6]:
ANNOTATIONS_DIR_PREFIX = "../data/Images"
DESTINATION_DIR = "../data/XML"
CLASS_MAPPING = {
    '0' : 'prohibitory',
    '1' : 'danger' ,
    '2' : 'mandatory' ,
    '3' : 'other' ,
}

In [7]:
def create_root(filename, width, height):
    root = ET.Element("annotation")
    ET.SubElement(root, "folder").text = "Images"
    ET.SubElement(root, "filename").text = (filename)
    ET.SubElement(root, "path").text = "../data/Images/{}".format(filename)
    source = ET.SubElement(root, "source")
    ET.SubElement(source, "database").text = "Unknown"
    size = ET.SubElement(root, "size")
    ET.SubElement(size, "width").text = str(width)
    ET.SubElement(size, "height").text = str(height)
    ET.SubElement(size, "depth").text = "3"
    ET.SubElement(root, "segmented").text = "0"
    return root
# ---------------------------------------------------
def create_object_annotation(root, voc):
    for ind,voc_label in voc.iterrows():
        obj = ET.SubElement(root, "object")
        ET.SubElement(obj, "name").text=str(CLASS_MAPPING.get(str(voc_label["id"])))
        ET.SubElement(obj, "pose").text = "Unspecified"
        ET.SubElement(obj, "truncated").text = str(0)
        ET.SubElement(obj, "difficult").text = str(0)
        bbox = ET.SubElement(obj, "bndbox")
        ET.SubElement(bbox, "xmin").text = str(voc_label["left"])
        ET.SubElement(bbox, "ymin").text = str(voc_label["top"])
        ET.SubElement(bbox, "xmax").text = str(voc_label["right"])
        ET.SubElement(bbox, "ymax").text = str(voc_label["bottom"])
    if(len(voc)==0):
        print(voc)
        print('no')
    return root
# ---------------------------------------------------
def create_file(filename, width, height, voc):
    root = create_root(filename, width, height)
    root = create_object_annotation(root, voc)
    tree = ET.ElementTree(root)
    tree.write("{}/{}.xml".format(DESTINATION_DIR, filename[:-4]))
# ---------------------------------------------------
def read_file(filename):
    img = cv.imread("{}/{}".format("../data/Images/", filename))
    w,h=img.shape[:2]
    voc = final_df.loc[final_df.path == filename[:-4]].copy()
    voc["name"] = CLASS_MAPPING.get(str((voc["id"])))
    create_file(filename, w, h, voc)
# ---------------------------------------------------
def start():
    if not os.path.exists(DESTINATION_DIR):
        os.makedirs(DESTINATION_DIR)
    for filename in os.listdir(ANNOTATIONS_DIR_PREFIX):
          if filename.endswith(".jpg"):
            read_file(filename)

In [8]:
start()

In [9]:
train_images = data_info_df[:700]['path']
valid_images = data_info_df[700:]['path']

In [10]:
with open('../data/train.txt','w') as train:
    for path in train_images:
        train.write(path)
        train.write('\n')

with open('../data/valid.txt','w') as val:
    for path in valid_images:
        val.write(path)
        val.write('\n')

# -----------------------------------
# Create TFRecord Files :

In [11]:
train_tfrecord_outputs = '../tfrecord/train/'
valid_tfrecord_outputs = '../tfrecord/valid/'
data_dir = '../data/'
classes_dir = '../data/obj.names'
split = 'train'

In [12]:
def _bytes_feature(value : list):
    return Feature(bytes_list=BytesList(value=value))
def _int64_feature(value : list):
    return Feature(int64_list=Int64List(value=value))
def _float_feature(value : list):
    return Feature(float_list=FloatList(value=value))

In [13]:
def create_example(annotations,class_map):
    image_path = os.path.join(data_dir,'Images',annotations['filename'])
    image_raw = open(image_path,'rb').read()
    key = hashlib.sha256(image_raw).hexdigest()
    width , height = int(annotations['size']['width']) , int(annotations['size']['height'])
    xmin , xmax , ymin , ymax , classes , classes_text , truncated , views , difficult_obj =\
    [] , [] , [] , [] , [] , [] , [] , [] ,[]
    if 'object' in annotations:
        for obj in annotations['obj']:
            difficult = bool(int(obj['difficult']))
            difficult_obj.append(int(difficult))
            xmin.append(float(obj['bndbox']['xmin']) / width)
            ymin.append(float(obj['bndbox']['ymin']) / height)
            xmax.append(float(obj['bndbox']['xmax']) / width)
            ymax.append(float(obj['bndbox']['ymax']) / height)
            classes_text.append(obj['name'].encode('utf8'))
            classes.append(class_map[obj['name']])
            truncated.append(int(obj['truncated']))
            views.append(obj['pose'].encode('utf8'))
    example = Example(features=Features(feature={
        'image/height' : _int64_feature(value=[height]),
        'image/width' : _int64_feature(value=[width]),
        'image/filename' : _bytes_feature(value=[annotations['filename'].encode('utf8')]),
        'image/source_id' : _bytes_feature(value=[annotations['filename'].encode('utf8')]),
        'image/sha256' : _bytes_feature(value=[key.encode('utf8')]),
        'image/encoded' : _bytes_feature(value=[image_raw.encode('utf8')]),
        'image/format' : _bytes_feature(value=['jpg'.encode('utf8')]),
        'image/object/bbox/xmin' : _float_feature(value=xmin),
        'image/object/bbox/xmax' : _float_feature(value=xmax),
        'image/object/bbox/ymin' : _float_feature(value=ymin),
        'image/object/bbox/ymax' : _float_feature(value=ymax),
        'image/object/class/text' : _bytes_feature(value=classes_text),
        'image/object/class/label' : _int64_feature(value=classes),
        'image/object/difficult' : _int64_feature(value=difficult_obj),
        'image/object/truncated' : _int64_feature(value=truncated),
        'image/object/views' : _bytes_feature(value=views)
    }))
    return example